In [6]:
import torch
from scripts.import_dataset import load_data_CSSD
from scripts.DiffusionModel import DiffusionModel
from scripts.Unet_valeria import UNet
from pathlib import Path

In [2]:
import os

script_dir = Path(os.path.abspath('')).resolve()
out_dir = script_dir / "out"

In [3]:
model = DiffusionModel(noise_predictor=UNet(3, 3))

In [4]:
ckpt_dict = torch.load(out_dir / "2022-07-20_13:43:49" / "checkpoint.pt")

In [5]:
model.load_state_dict(ckpt_dict["parameters"])

<All keys matched successfully>

In [11]:
data, _ = load_data_CSSD(1, 1, 0.5)
x = next(iter(data))
img = model.generate_from(torch.randn_like(x))[0]

In [14]:
import torchvision.transforms as T

T.ToPILImage()(img).show()
T.ToPILImage()(x[0]).show()

In [15]:
from scripts.trainer import Trainer

trainer = Trainer(model=model,
                  out_path=script_dir / "out" / "ipynb")
train_loader, test_loader = load_data_CSSD(train_batch_size=32,
                                           test_batch_size=64,
                                           ratio_test=0.2)
trainer.train(n_epochs=10,
              train_dataloader=train_loader,
              val_dataloader=test_loader,
              valid_each=2,
              )

Training epoch:   0%|          | 0/10 [00:00<?, ?it/s]

In [16]:
img = model.generate_from(torch.randn_like(x))[0]
T.ToPILImage()(img).show()

In [21]:
noised_x, noise, t = model._noise_generator.add_noise(x, torch.tensor(99, dtype=torch.long))
img = model.generate_from(noised_x)[0]
T.ToPILImage()(img).show()